# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1.
사후확률 : 모든 특성의 우도의 곱 * 사전확률 / 증거

### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [3]:
# pip install pandas
import pandas as pd
import re

In [4]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [5]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [9]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

0                     love great exceeded expectations
1     worst purchase have ever made completely useless
2      average nothing special but not terrible either
3    great who can help but love design highly reco...
4    terrible experience will never buy from poor b...
5    acceptable but expected better not just accept...
6       absolutely wonderful very satisfied with great
7     quality poor broke after one use terrible enough
8    acceptable for price but there better options ...
9      great quality fast shipping with wonderful love
Name: review, dtype: object


기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다. 

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다. 

In [10]:
# 사전 확률 구하는 코드를 작성해주세요.
# 클래스별 빈도 계산
class_counts = df['sentiment'].value_counts()

# 전체 리뷰 수
total_reviews = len(df)

# 사전 확률 계산
P_positive = class_counts['positive'] / total_reviews
P_negative = class_counts['negative'] / total_reviews
P_neutral = class_counts['neutral'] / total_reviews

# 결과 출력
print(f"P(positive) = {P_positive}")
print(f"P(negative) = {P_negative}")
print(f"P(neutral) = {P_neutral}")

P(positive) = 0.4
P(negative) = 0.3
P(neutral) = 0.3


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [39]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [40]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [41]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# 리뷰 데이터를 단어 벡터로 변환
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()

# 단어 리스트 추출
words = vectorizer.get_feature_names_out()

# 단어 빈도 테이블 생성
frequency_matrix = pd.DataFrame(review_array, columns=words)
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)

# 각 단어의 조건부 확률을 구하는 함수
def calculate_conditional_probabilities(frequency_matrix, word, sentiment_class):
    # 해당 클래스의 빈도 계산
    class_df = frequency_matrix[frequency_matrix['sentiment'] == sentiment_class]
    
    # 해당 클래스에서의 단어 빈도 합계
    if word in class_df.columns:
        class_word_sum = class_df[word].sum()
    else:
        class_word_sum = 0  # 단어가 해당 클래스에 없을 경우 0으로 설정
    
    # 해당 클래스 내 모든 단어 빈도 합계
    class_total_word_sum = class_df[words].sum().sum()
    
    # 라플라스 스무딩을 적용하여 조건부 확률 계산
    conditional_probability = (class_word_sum + 1) / (class_total_word_sum + len(words))
    return conditional_probability

# 첫 번째 문장의 단어들에 대한 조건부 확률 계산
words_to_check = ['love', 'great', 'awesome']

# 각 단어의 조건부 확률을 계산
for word in words_to_check:
    P_word_pos = calculate_conditional_probabilities(frequency_matrix, word, 'positive')
    P_word_neg = calculate_conditional_probabilities(frequency_matrix, word, 'negative')
    P_word_neu = calculate_conditional_probabilities(frequency_matrix, word, 'neutral')
    
    print(f"P({word}|positive) = {P_word_pos}")
    print(f"P({word}|negative) = {P_word_neg}")
    print(f"P({word}|neutral) = {P_word_neu}")
    print("")



P(love|positive) = 0.05
P(love|negative) = 0.01282051282051282
P(love|neutral) = 0.01282051282051282

P(great|positive) = 0.0625
P(great|negative) = 0.01282051282051282
P(great|neutral) = 0.01282051282051282

P(awesome|positive) = 0.0125
P(awesome|negative) = 0.01282051282051282
P(awesome|neutral) = 0.01282051282051282



독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [14]:
# 가능도 구하는 코드를 작성해주세요.
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# 리뷰 데이터를 단어 벡터로 변환
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()

# 단어 리스트 추출
words = vectorizer.get_feature_names_out()

# 단어 빈도 테이블 생성
frequency_matrix = pd.DataFrame(review_array, columns=words)
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)

# 각 단어의 조건부 확률을 구하는 함수
def calculate_conditional_probabilities(frequency_matrix, word, sentiment_class):
    # 해당 클래스의 빈도 계산
    class_df = frequency_matrix[frequency_matrix['sentiment'] == sentiment_class]
    
    # 해당 클래스에서의 단어 빈도 합계
    if word in class_df.columns:
        class_word_sum = class_df[word].sum()
    else:
        class_word_sum = 0  # 단어가 해당 클래스에 없을 경우 0으로 설정
    
    # 해당 클래스 내 모든 단어 빈도 합계
    class_total_word_sum = class_df[words].sum().sum()
    
    # 라플라스 스무딩을 적용하여 조건부 확률 계산
    conditional_probability = (class_word_sum + 1) / (class_total_word_sum + len(words))
    return conditional_probability

# Naive Bayes 독립성 가정을 이용한 가능도 계산 함수
def calculate_likelihood(words_to_check, sentiment_class):
    likelihood = 1.0
    for word in words_to_check:
        likelihood *= calculate_conditional_probabilities(frequency_matrix, word, sentiment_class)
    return likelihood

# 첫 번째 문장의 단어들에 대한 가능도 계산
words_to_check = ['love', 'great', 'awesome']

# 각 클래스에 대한 가능도 계산
likelihood_pos = calculate_likelihood(words_to_check, 'positive')
likelihood_neg = calculate_likelihood(words_to_check, 'negative')
likelihood_neu = calculate_likelihood(words_to_check, 'neutral')

# 결과 출력
print(f"P(love, great, awesome | positive) = {likelihood_pos}")
print(f"P(love, great, awesome | negative) = {likelihood_neg}")
print(f"P(love, great, awesome | neutral) = {likelihood_neu}")











P(love, great, awesome | positive) = 3.906250000000001e-05
P(love, great, awesome | negative) = 2.107250627960687e-06
P(love, great, awesome | neutral) = 2.107250627960687e-06


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [15]:
import numpy as np
# 사전 확률 구하기
def calculate_prior_probability(sentiment_class):
    class_count = frequency_matrix[frequency_matrix['sentiment'] == sentiment_class].shape[0]
    total_count = frequency_matrix.shape[0]
    return class_count / total_count

# Naive Bayes 독립성 가정을 이용한 가능도 계산 함수
def calculate_likelihood(words_to_check, sentiment_class):
    likelihood = 1.0
    for word in words_to_check:
        likelihood *= calculate_conditional_probabilities(frequency_matrix, word, sentiment_class)
    return likelihood

# 전체 확률 P(data)를 구하는 함수
def calculate_total_probability(words_to_check):
    total_probability = 0
    for sentiment_class in ['positive', 'negative', 'neutral']:
        prior = calculate_prior_probability(sentiment_class)
        likelihood = calculate_likelihood(words_to_check, sentiment_class)
        total_probability += prior * likelihood
    return total_probability

# 사후 확률 P(class | data) 계산 함수
def calculate_posterior(words_to_check, sentiment_class):
    prior = calculate_prior_probability(sentiment_class)
    likelihood = calculate_likelihood(words_to_check, sentiment_class)
    total_prob = calculate_total_probability(words_to_check)
    posterior = (prior * likelihood) / total_prob
    return posterior

# 첫 번째 문장의 단어들
words_to_check_1 = ['love', 'great', 'awesome']

# 첫 번째 문장에 대한 각 클래스의 사후 확률 계산
P_pos_1 = calculate_posterior(words_to_check_1, 'positive')
P_neg_1 = calculate_posterior(words_to_check_1, 'negative')
P_neu_1 = calculate_posterior(words_to_check_1, 'neutral')

print(f"P(positive | target_review1) = {P_pos_1}")
print(f"P(negative | target_review1) = {P_neg_1}")
print(f"P(neutral | target_review1) = {P_neu_1}")

# 두 번째 문장의 단어들
words_to_check_2 = ['terrible', 'not', 'never']

# 두 번째 문장에 대한 각 클래스의 사후 확률 계산
P_pos_2 = calculate_posterior(words_to_check_2, 'positive')
P_neg_2 = calculate_posterior(words_to_check_2, 'negative')
P_neu_2 = calculate_posterior(words_to_check_2, 'neutral')

print(f"P(positive | target_review2) = {P_pos_2}")
print(f"P(negative | target_review2) = {P_neg_2}")
print(f"P(neutral | target_review2) = {P_neu_2}")



P(positive | target_review1) = 0.9251391943105787
P(negative | target_review1) = 0.0374304028447106
P(neutral | target_review1) = 0.0374304028447106
P(positive | target_review2) = 0.09336884305364711
P(negative | target_review2) = 0.4533155784731765
P(neutral | target_review2) = 0.4533155784731765


A2-1.   
Target review1의 분류 결과:   positive가 92퍼센트
Target review2의 분류 결과:   Negetive, neutral이 각각 45퍼센트 

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2. 특정 단어(awesome)이 관측되지 않아 오류가 발생했다. 라플라스 스무딩을 통해 분자를 1로 만들어주어 그러한 부작용을 없앤다.